# Initialization

In [1]:
import os, shutil, importlib
import utils; importlib.reload(utils)

# os.environ["CUDA_VISIBLE_DEVICES"]=''
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import h5py
import keras
import sklearn

from glob import glob
from PIL import Image
from os.path import join as jp

from keras import models, optimizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.layers import *
from keras.optimizers import Adam
from keras import regularizers

from scipy import ndimage

from sklearn import metrics
from sklearn.model_selection import train_test_split

from IPython.display import FileLink

%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

Using TensorFlow backend.


## Struct Dir

For this comp, need to stratify the valid set based on the drivers as well as the categories

In [4]:
path = '../data/statefarm/'
cats = pd.read_csv('../data/statefarm/driver_imgs_list.csv').set_index('img')

In [27]:
utils.create_dirs(path, cats.classname.unique())

Move 20% of each class, stratified by subject, into valid

In [28]:
for cl in cats.classname.unique():
    imgs_train, imgs_valid = train_test_split(cats[cats.classname == cl].index,
                                              test_size=0.2,
                                              stratify=cats[cats.classname == cl].subject)
    for img in imgs_valid:
        cats.ix[img, 'split'] = 'valid'
        shutil.move(os.path.join(path, 'train', cl, img),
                    os.path.join(path, 'valid', cl))
cats.split.fillna('train', inplace=True)

Copy about 100 from each class into sample

In [56]:
for cl in cats.classname.unique():
    cats_train = cats[(cats.classname == cl) & (cats.split == 'train')]
    cats_valid = cats[(cats.classname == cl) & (cats.split == 'valid')]
    
    # Remove any subjects that only have 1 occurence in either set
    cats_valid_cnts = cats_valid.subject.value_counts()
    cats_train_cnts = cats_train.subject.value_counts()
    cats_valid_keep = cats_valid_cnts[cats_valid_cnts > 1].index.values
    cats_train_keep = cats_train_cnts[cats_train_cnts > 1].index.values
    cats_valid = cats_valid[cats_valid.subject.isin(cats_valid_keep)]    
    cats_train = cats_train[cats_train.subject.isin(cats_train_keep)]    
    
    # Split data
    _, imgs_train = train_test_split(cats_train.index,
                                     test_size=100,
                                     stratify=cats_train.subject)
    _, imgs_valid = train_test_split(cats_valid.index,
                                     test_size=100,
                                     stratify=cats_valid.subject)
    for img_tr, img_v in zip(imgs_train, imgs_valid):
        shutil.copy(jp(path, 'train', cl, img_tr),
                    jp(path, 'sample', 'train', cl, img_tr))
        shutil.copy(jp(path, 'valid', cl, img_v),
                    jp(path, 'sample', 'valid', cl, img_v))

100 100
100 100
100 100
100 100
100 100
100 100
100 100
100 100
100 100
100 100


Copy test data

In [73]:
files = np.random.choice(glob(jp(path, 'test', 'test', '*.jpg')), 100, replace=False)
for img in files:
    shutil.copy(img, jp(path, 'sample', 'test', 'test', img.split('/')[-1]))

## Define Paths

In [2]:
# path = '../data/statefarm/'
path = '../data/statefarm/sample/'
path_model = '../data/statefarm/models/'
path_test = path + 'test'
path_train = path + 'train'
path_valid = path + 'valid'
path_img_arrays = path + 'img_arrays/'

path_submit = path_model + 'submissions/'
path_checkpoint = path_model + 'checkpoints/'
path_results = path_model + 'results/'

batch_size = 8

# Basic Models

## Create Batches

In [3]:
batch_size = 8

In [4]:
batches_train = utils.get_batches(path+'train', batch_size=batch_size)
batches_valid = utils.get_batches(path+'valid', batch_size=batch_size, shuffle=False)

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [5]:
classes, filenames = utils.get_classes(path)

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 100 images belonging to 1 classes.


## Linear Model

In [98]:
model_linear = models.Sequential([
        BatchNormalization(axis=3, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax')     
    ])

In [99]:
model_linear.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [100]:
model_linear.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=5, callbacks=None)

Epoch 1/5
1000/1000 [==============================] - 31s - loss: 14.3575 - acc: 0.1030 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/5
1000/1000 [==============================] - 25s - loss: 14.3873 - acc: 0.1060 - val_loss: 9.7031 - val_acc: 0.3980
Epoch 3/5
1000/1000 [==============================] - 25s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.9737 - val_acc: 0.0710
Epoch 4/5
1000/1000 [==============================] - 25s - loss: 14.4096 - acc: 0.1060 - val_loss: 15.8901 - val_acc: 0.0130
Epoch 5/5
1000/1000 [==============================] - 25s - loss: 14.4096 - acc: 0.1060 - val_loss: 16.0386 - val_acc: 0.0040


Model isnt doing anything, lower lr

In [102]:
model_linear = models.Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax')     
    ])
model_linear.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model_linear.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=2, callbacks=None)

Epoch 1/2
1000/1000 [==============================] - 26s - loss: 2.3173 - acc: 0.2490 - val_loss: 1.8748 - val_acc: 0.3430
Epoch 2/2
1000/1000 [==============================] - 25s - loss: 1.3051 - acc: 0.5790 - val_loss: 1.2341 - val_acc: 0.5570


Increase lr 

In [103]:
K.set_value(model_linear.optimizer.lr, 0.0001)

In [105]:
model_linear.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=4, callbacks=None)

Epoch 1/4
1000/1000 [==============================] - 26s - loss: 3.0042 - acc: 0.5600 - val_loss: 1.8723 - val_acc: 0.6950
Epoch 2/4
1000/1000 [==============================] - 25s - loss: 1.4681 - acc: 0.7440 - val_loss: 1.9966 - val_acc: 0.7310
Epoch 3/4
1000/1000 [==============================] - 25s - loss: 0.8713 - acc: 0.8410 - val_loss: 1.0877 - val_acc: 0.8224
Epoch 4/4
1000/1000 [==============================] - 25s - loss: 0.4035 - acc: 0.8980 - val_loss: 0.8628 - val_acc: 0.8250


acc_valid stablizes around 82. Generate random valid batches to see if there is sampling bias

In [106]:
batches_rand = utils.get_batches(jp(path, 'valid'), batch_size=batch_size, shuffle=True)

Found 1000 images belonging to 10 classes.


In [115]:
val_scores = [model_linear.evaluate_generator(batches_rand, batches_rand.n/10)
              for i in range(10)]

In [116]:
np.round(val_scores, 2)

array([[ 1.21,  0.79],
       [ 0.89,  0.84],
       [ 0.83,  0.83],
       [ 1.09,  0.82],
       [ 1.14,  0.81],
       [ 1.06,  0.79],
       [ 1.57,  0.78],
       [ 0.91,  0.77],
       [ 0.81,  0.81],
       [ 1.12,  0.81]])

## Linear Model with Maxout

In [117]:
model_linear_mo = models.Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        MaxoutDense(10),
        Activation('softmax')
    ])
model_linear_mo.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model_linear_mo.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=2, callbacks=None)

Epoch 1/2
1000/1000 [==============================] - 26s - loss: 6.7963 - acc: 0.1230 - val_loss: 5.0418 - val_acc: 0.2570
Epoch 2/2
1000/1000 [==============================] - 25s - loss: 3.8167 - acc: 0.2760 - val_loss: 3.2944 - val_acc: 0.3460


In [119]:
K.set_value(model_linear.optimizer.lr, 0.0001)

In [121]:
model_linear_mo.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=5, callbacks=None)

Epoch 1/5
1000/1000 [==============================] - 25s - loss: 1.5415 - acc: 0.5800 - val_loss: 2.7848 - val_acc: 0.4222
Epoch 2/5
1000/1000 [==============================] - 24s - loss: 0.9965 - acc: 0.7110 - val_loss: 1.8750 - val_acc: 0.5000
Epoch 3/5
1000/1000 [==============================] - 24s - loss: 0.6589 - acc: 0.8110 - val_loss: 1.8518 - val_acc: 0.5450
Epoch 4/5
1000/1000 [==============================] - 25s - loss: 0.4296 - acc: 0.8770 - val_loss: 1.4340 - val_acc: 0.6210
Epoch 5/5
1000/1000 [==============================] - 25s - loss: 0.2806 - acc: 0.9370 - val_loss: 1.6638 - val_acc: 0.6220


## L2 regularization

In [128]:
model_linear_l2 = models.Sequential([
        BatchNormalization(axis=3, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=regularizers.l2(l=0.01))     
    ])
model_linear_l2.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_linear_l2.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=5, callbacks=None)

Epoch 1/5
1000/1000 [==============================] - 26s - loss: 10.3370 - acc: 0.1920 - val_loss: 11.0494 - val_acc: 0.2330
Epoch 2/5
1000/1000 [==============================] - 25s - loss: 9.5793 - acc: 0.2880 - val_loss: 10.3611 - val_acc: 0.3430
Epoch 3/5
1000/1000 [==============================] - 25s - loss: 9.3116 - acc: 0.3430 - val_loss: 10.8533 - val_acc: 0.1856
Epoch 4/5
1000/1000 [==============================] - 25s - loss: 6.9767 - acc: 0.4680 - val_loss: 7.9728 - val_acc: 0.4760
Epoch 5/5
1000/1000 [==============================] - 25s - loss: 5.8017 - acc: 0.5820 - val_loss: 1.4761 - val_acc: 0.7570


## Single Conv Layer

In [12]:
def gen_model_conv(batches, epochs2=4):
    model_conv = models.Sequential([
            BatchNormalization(axis=3, input_shape=(224, 224, 3)),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Flatten(),
            MaxoutDense(100),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    model_conv.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model_conv.fit_generator(batches, samples_per_epoch=batches.n,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n,
                         nb_epoch=2, callbacks=None)
    K.set_value(model_conv.optimizer.lr, 1e-3)
    model_conv.fit_generator(batches, samples_per_epoch=batches.n,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n,
                         nb_epoch=epochs2, callbacks=None)
    return model_conv

In [13]:
model_conv = gen_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 57s - loss: 1.9145 - acc: 0.3880 - val_loss: 3.1522 - val_acc: 0.1220
Epoch 2/2
1000/1000 [==============================] - 55s - loss: 0.4965 - acc: 0.9000 - val_loss: 2.8061 - val_acc: 0.1890
Epoch 1/4
1000/1000 [==============================] - 60s - loss: 1.1152 - acc: 0.6550 - val_loss: 1.1022 - val_acc: 0.5590
Epoch 2/4
1000/1000 [==============================] - 61s - loss: 0.2055 - acc: 0.9650 - val_loss: 0.7434 - val_acc: 0.7510
Epoch 3/4
1000/1000 [==============================] - 62s - loss: 0.0418 - acc: 0.9980 - val_loss: 0.3475 - val_acc: 0.9060
Epoch 4/4
1000/1000 [==============================] - 64s - loss: 0.0155 - acc: 0.9980 - val_loss: 0.2117 - val_acc: 0.9480


Conv model trains very quickly. Add regularization

## Data Augmentation

Test different data augmentations to find best

#### Width Shift

In [14]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [15]:
model = gen_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 55s - loss: 2.4474 - acc: 0.2230 - val_loss: 2.4816 - val_acc: 0.1650
Epoch 2/2
1000/1000 [==============================] - 56s - loss: 1.5380 - acc: 0.5050 - val_loss: 1.8161 - val_acc: 0.2910
Epoch 1/4
1000/1000 [==============================] - 59s - loss: 1.8985 - acc: 0.3780 - val_loss: 1.4240 - val_acc: 0.5780
Epoch 2/4
1000/1000 [==============================] - 61s - loss: 1.2066 - acc: 0.6200 - val_loss: 1.1095 - val_acc: 0.6370
Epoch 3/4
1000/1000 [==============================] - 62s - loss: 0.8812 - acc: 0.7220 - val_loss: 0.7807 - val_acc: 0.7390
Epoch 4/4
1000/1000 [==============================] - 63s - loss: 0.7060 - acc: 0.7890 - val_loss: 0.6498 - val_acc: 0.7920
